## Dependencies

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from datetime import date
import csv
import time
import zipfile
import requests
import steam
from steam.steamid import SteamID
from steam.webapi import WebAPI, get
import steamleaderboards as sl
import os,sys,stat
from zipfile import ZipFile, ZipInfo
import difflib
import re

# 1. unpack stats from folder

(from zip)

In [2]:
# App will find directory on computer or user can input the directory of the stats folder
#dataDir = 'G:\Steam\steamapps\common\FPSAimTrainer\FPSAimTrainer\stats_comp.zip'
#target = ['Kills:', 'Avg TTK:', 'Score:', 'Scenario:']
#temp_folder = '../temp'
#
## unzip and extract to a temporary folder
#with zipfile.ZipFile(dataDir, 'r') as zip_ref:
#   zip_ref.extractall(temp_folder) # If files with same filename are already present at extraction location then it will overwrite those files.
#

In [3]:
#unzipped_stats = temp_folder + '\stats'
#
#try:
#    stats = pd.read_csv(unzipped_stats)
#except:
#    stats = pd.DataFrame(columns=['Kills', 'Avg TTK', 'Score', 'Scenario'])

FROM DIRECTORY

In [4]:
dataDir = 'G:\Steam\steamapps\common\FPSAimTrainer\FPSAimTrainer\stats'
target = ['Kills:', 'Avg TTK:', 'Score:', 'Scenario:']

try:
    stats = pd.read_csv(dataDir + '\\stats.csv')
except:
    stats = pd.DataFrame(columns=['Kills', 'Avg TTK', 'Score', 'Scenario','Damage Done'])

In [5]:
# Empty dict to store our csv data
data = {}

# Go through all the files and store our target data in our empty dictionary, including the datetime of the file.
for filename in os.listdir(dataDir):
    
    f = open(f'{dataDir}/{filename}', 'r')
    with f as csv_file:
        csv_reader = csv.reader(csv_file)
        for line in csv_reader:
            try:
                if line[0] in target:
                    data[(line[0][:-1])] = line[1]     
            except IndexError:
                continue
        f.close()

    scenario_date = f'{dataDir}/{filename}'[-29:-10]
    data["Date"] = scenario_date
    df = pd.DataFrame(data, index=[0])
    stats = pd.concat([stats,df]).drop_duplicates().reset_index(drop=True)

# CLEAN


In [6]:
stats.head(3)

,Kills,Avg TTK,Score,Scenario,Date
0,2,0,10792.587891,180 Regen Warzone,2021.07.27-22.47.26
1,12,0,22690.832031,180 Regen Warzone,2021.07.27-22.48.41
2,7,0,15467.339844,180 Regen Warzone,2021.07.27-22.49.53


In [7]:
stats.isna().sum() #NO Nan

Kills       0
Avg TTK     0
Score       0
Scenario    0
Date        0
dtype: int64

In [8]:
stats.duplicated().sum() #NO duplicates

0

In [9]:
stats['Kills'] = pd.to_numeric(stats['Kills'], downcast='integer')
stats['Avg TTK'] = pd.to_numeric(stats['Avg TTK'], downcast='float')
stats['Score'] = pd.to_numeric(stats['Score'], downcast='float')
stats['Date'] = pd.to_datetime(stats['Date'], yearfirst=True, format=('%Y.%m.%d-%H.%M.%S'),exact=False)

In [10]:
stats.head()

,Kills,Avg TTK,Score,Scenario,Date
0,2,0.0,10792.587891,180 Regen Warzone,2021-07-27 22:47:26
1,12,0.0,22690.832031,180 Regen Warzone,2021-07-27 22:48:41
2,7,0.0,15467.339844,180 Regen Warzone,2021-07-27 22:49:53
3,9,0.0,16614.562500,180 Regen Warzone,2021-07-28 22:45:19
4,10,0.0,16942.285156,180 Regen Warzone,2021-07-28 22:46:26


# SAVE

In [11]:
stats.to_csv("data/stats.csv", index=False)

# 2. CREATE COUNTER THAT COUNTS UP MY HIGHEST SCORING GAME IN REAL-TIME

### Search Bar

In [37]:

scenario_names = (stats['Scenario'].unique()).astype(str)
user_input = '180 warzon' #tkinter search bar
try:
    closest_match = (difflib.get_close_matches(str(user_input), scenario_names, n=1,cutoff=0.45).pop(0))
    print(closest_match)
except Exception:
    print('we can\'t find that scenario. Did u spel it rite?')


180 Regen Warzone


In [38]:
scenario = stats.loc[stats['Scenario'] == closest_match]
scenario

,Kills,Avg TTK,Score,Scenario,Date
0,2,0.0,10792.587891,180 Regen Warzone,2021-07-27 22:47:26
1,12,0.0,22690.832031,180 Regen Warzone,2021-07-27 22:48:41
2,7,0.0,15467.339844,180 Regen Warzone,2021-07-27 22:49:53
3,9,0.0,16614.562500,180 Regen Warzone,2021-07-28 22:45:19
4,10,0.0,16942.285156,180 Regen Warzone,2021-07-28 22:46:26
5,4,0.0,7884.500000,180 Regen Warzone,2021-07-29 17:31:04
6,5,0.0,9950.264648,180 Regen Warzone,2021-07-29 17:32:49
7,12,0.0,16046.000000,180 Regen Warzone,2021-07-30 18:12:18
8,6,0.0,10583.379883,180 Regen Warzone,2021-07-30 18:13:24
9,9,0.0,13275.500000,180 Regen Warzone,2021-07-30 18:42:56


### Date of highest score

In [39]:


high_score_date = []
# The following is simply to catch the high_score_date, nothing more!
if sum(scenario['Score']) == 0.0:
    high_score_date.append(scenario.sort_values('Kills')['Date'].iloc[-1])
elif sum(scenario['Kills']) > 0  and sum(scenario['Score']) > 0.0:
    high_score_date.append(scenario.sort_values('Score')['Date'].iloc[-1])

hs_date = str(high_score_date[0])
hs_date

'2021-11-18 17:49:23'

### Now we search for the .csv file with the highest score

In [40]:
hs_year = hs_date[:4]
hs_month = hs_date[5:7]
hs_day = hs_date[8:10]
hs_hour = hs_date[11:13]
hs_min = hs_date[14:16]
hs_sec = hs_date[17:19]

In [41]:
Kill = []
Time = []
Acc = []
Challenge_start = []

for filename in os.listdir(dataDir):
    if f'{hs_year}.{hs_month}.{hs_day}-{hs_hour}.{hs_min}.{hs_sec} Stats.csv' in filename:
        f = open(f'{dataDir}/{filename}', 'r')
        with f as csv_file:
            csv_reader = csv.reader(csv_file)
            for line in csv_reader:
                try:
                    Kill.append(line[0])
                    Time.append(line[1])
                    Acc.append(line[7])
                except Exception:
                    break 
            for line in csv_reader:
                try:
                    if line[0] in 'Challenge Start:' and line[1] not in 'H:M:S.s':
                        Challenge_start.append(line[1])
                    elif line[0] in 'Challenge Start:' and line[1] in 'H:M:S.s' or '':
                        start = pd.to_datetime(Time[-1]) - ((pd.to_datetime(Time[-1]) - pd.to_datetime(Time[1])).round(freq='T')) # deduce length between challenge start and first kill
                        Challenge_start.append(str(start)[11:-3])
                except Exception:
                    continue              
        f.close()
Challenge_start

['17:48:23.911']

In [42]:
data_tuples = list(zip(Kill,Time,Acc))

high_score_df = (pd.DataFrame(data_tuples,columns=('Kill #','Time', 'Accuracy'))
                    .replace(['Kill #','Accuracy'], 0)
                    ).replace('Timestamp',''.join(str(Challenge_start[0])))

high_score_df.head()

,Kill #,Time,Accuracy
0,0,17:48:23.911,0
1,1,17:48:28.853,0.361
2,2,17:48:30.105,0.562
3,3,17:48:32.250,0.343
4,4,17:48:33.564,0.5


CLEAN (i need the dates column type to be float in order to make my calculations)

In [43]:
high_score_df['Kill #'] = pd.to_numeric(high_score_df['Kill #'], downcast='integer')
high_score_df['Accuracy'] = pd.to_numeric(high_score_df['Accuracy'], downcast='float')
# DO I WANY TIME IN DATE TIME FORMAT OR EASIER AS STRING??
high_score_df['Time'] = pd.to_datetime(high_score_df['Time'],format=('%H:%M:%S.%f'),exact=False)
high_score_df['ms'] = (high_score_df['Time'] - high_score_df['Time'].iloc[0]) / pd.to_timedelta('1ms')
high_score_df

,Kill #,Time,Accuracy,ms
0,0,1900-01-01 17:48:23.911,0.000,0.0
1,1,1900-01-01 17:48:28.853,0.361,4942.0
2,2,1900-01-01 17:48:30.105,0.562,6194.0
3,3,1900-01-01 17:48:32.250,0.343,8339.0
4,4,1900-01-01 17:48:33.564,0.500,9653.0
5,5,1900-01-01 17:48:37.044,0.308,13133.0
6,6,1900-01-01 17:48:37.839,0.556,13928.0
7,7,1900-01-01 17:48:38.883,0.357,14972.0
8,8,1900-01-01 17:48:40.782,0.409,16871.0
9,9,1900-01-01 17:48:41.857,0.583,17946.0


## create counter.

In [48]:
score_kills = 0

for index, value in enumerate(high_score_df['ms']):
    if value == 0.0:
        print('GO')      
        time.sleep((high_score_df['ms'][1])/1000)
        score_kills += 1
        print(f'Score/Kills: {score_kills}',end='\r')
    else:
        try:
            time.sleep((high_score_df['ms'][index+1] - high_score_df['ms'][index])/1000)
            score_kills += 1 
            print(f'Score/Kills: {score_kills}',end='\r')
        except Exception:
            break

GO


KeyboardInterrupt: 